In [10]:
import pandas as pd
from openpyxl import load_workbook
import locale
import re
import datetime
import numpy as np

locale.setlocale(locale.LC_ALL, 'Swedish_Sweden.1252')

def new_sheetsNames(start=0, end=0):
    file = 'Privat_budget_210921 - 3 - row added.xlsx'
    newfile = 'Privat_budget_210921 - NEW.xlsx'

    wb = load_workbook(file)

    startYear = 2020
    months = ['Januari', 'Februari', 'Mars', 'April', 'Maj', 'Juni', 'Juli', 'Augusti', 'September', 'Oktober', 'November', 'December']
    
    for sheet in wb.sheetnames:
        _sheet = wb[sheet]
        if '25e' in sheet:
            
            for j in range(len(months)): 
                if str(months[j]) in sheet:
                    
                    date = f'25 {months[j]} {str(startYear)}'
                    _sheet.title = date
                    wb.save(newfile)
                    if j == len(months)-1:
                        startYear += 1
                        j = 0
            
    print('Alla kolumnnamn ändrade...')

def char_search(string): 
    if re.search(':e', string):
        string = re.sub(':e', '', string)
        return datetime.datetime.strptime(string, '%d %B %Y').date()
    return datetime.datetime.strptime(string, '%d %B %Y').date()

def summa(x):
    pass

def make_df():

    budget = pd.ExcelFile("Privat_budget_210921 - NEW.xlsx")
    df = {}
    concat  = {}  
    df_list = []
    peter_drop_list = ["Spara Bilen", "Inkomst", "Gem. Räk. TOT"]

    print('Reading file...')
    for sheet in budget.sheet_names:
        if "25" in sheet:
            df = pd.read_excel(budget, sheet, usecols="A:D", nrows=22, converters={'Date': datetime})
            df.insert(0, 'Date', char_search(sheet))
            df['Date'] = pd.to_datetime(df['Date'])
            df_list.append(df)
            concat = pd.concat(df_list, ignore_index=True)
            
            df2021 = concat[concat['Date'].dt.to_period('Y') == '2021']
            df2022 = concat[concat['Date'].dt.to_period('Y') == '2022']
            
            df2021xxx = df2021.join(df2021.groupby('Peter')['Unnamed: 1'].sum(), on='Peter', rsuffix='XXX')
            
            
            #df2021['Sum 2021'] = df2021.groupby('Peter')['Unnamed: 1'].transform('sum')
            #df2022['Sum 2022'] = df2022.groupby('Peter')['Unnamed: 1'].transform('sum')
            
            df2021_grouped = df2021[['Peter', 'Unnamed: 1']].groupby('Peter').agg(['count', 'mean', 'sum'])
            df2022_grouped = df2022[['Peter', 'Unnamed: 1']].groupby('Peter').agg(['count', 'mean', 'sum'])
            
           
            
            peter = concat[['Date', 'Peter', 'Unnamed: 1']].groupby('Peter').agg(['count', 'sum', 'mean'])
            peter1 = peter.drop(peter_drop_list)
            sara = concat[['Date', 'Sara', 'Unnamed: 3']].groupby('Date').agg(['count', 'sum', 'mean'])
            # merged = pd.concat([peter, sara], axis=1)
            
            year = peter1.groupby(concat['Date'].dt.year).agg(['sum', 'mean'])
            
            df99 = concat.loc[(concat['Date'] == 2021-01-25)]
            
            filtered_dt = df99.groupby('Peter').filter(lambda x: (x['2021'].str.startswith('Hyra')))

    print(filtered_dt)
    
                    
if __name__ == '__main__':
    make_df()



SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (3780220009.py, line 82)